In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

import bb_rt_helper

In [ ]:
import yaml

In [ ]:
with open('refined_corridors_hotspots.yaml', 'r') as f:
    all_corr_hs = yaml.safe_load(f)

In [ ]:
## future parameter
district_param = '07 - Los Angeles'

In [ ]:
all_corr_hs

In [ ]:
# find district information matching parameter
for district in all_corr_hs['districts']:
    if district['district'] == district_param:
        district_dict = district

In [ ]:
district_dict

In [ ]:
fm_dict = {}
corridors = []
hotspots = []
# override default date if needed (so far only d3/SacRT)
analysis_date = dt.date.fromisoformat(district_dict['date_override']) if district_dict['date_override'] else dt.date(2022, 5, 4)

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist=district_param, category='corridor',
                                       get_sorted = district_dict['get_sorted_corr'],
                                       analysis_date=analysis_date, fm_dict = fm_dict)

In [ ]:
# save speedmap views across all operators for future combined map
corr_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    corr_speed_views += [fm_dict[key].detailed_map_view.copy()]

In [ ]:
district_dict['corridors'][0]['corridor']

In [ ]:
def parse_corridor(corridor_dict):
    '''
    Construct analysis corridor and calculate corridor metrics for each corridor
    specified. Also, add corridor information to overall list for future map/export
    
    corridor_dict: dict, item['corridor'] for any item in district_dict['corridors']
    '''
    global corridors
    subcorridors = []
    for part in corridor_dict['parts']:
        fm_dict[corridor_dict['itp_id']].autocorridor(part['shape_id'], part['stop_bounds'])
        _corridor = fm_dict[corridor_dict['itp_id']].corridor.copy()
        subcorridors += [_corridor]
    full_corridor = pd.concat(subcorridors).dissolve()
    fm_dict[corridor_dict['itp_id']].add_corridor(full_corridor)
    if corridor_dict['route_filter']:
        fm_dict[corridor_dict['itp_id']].set_filter(route_names = corridor_dict['route_filter'])
    else:
        fm_dict[corridor_dict['itp_id']].reset_filter()
    fm_dict[corridor_dict['itp_id']].corridor_metrics()
    display(fm_dict[corridor_dict['itp_id']].quick_map_corridor())
    measured_corridor = fm_dict[corridor_dict['itp_id']].corridor.copy()
    corridors += [measured_corridor]
    return

In [ ]:
# parse_corridor(district_dict['corridors'][0]['corridor'])

In [ ]:
for corridor in district_dict['corridors']:
    parse_corridor(corridor['corridor'])

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist=district_param, category='hotspot',
                                       get_sorted = district_dict['get_sorted_hs'],
                                       analysis_date=analysis_date, fm_dict = fm_dict)

In [ ]:
hs_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    hs_speed_views += [fm_dict[key].detailed_map_view.copy()]

In [ ]:
def parse_hotspot(hotspot_dict):
    '''
    Construct analysis corridor and calculate corridor metrics for each hotspot
    specified. Also, add hotspot information to overall list for future map/export
    
    hotspot_dict: dict, item['hotspot'] for any item in district_dict['hotspots']
    '''
    global hotspots
    subcorridors = []
    for part in hotspot_dict['parts']:
        fm_dict[hotspot_dict['itp_id']].autocorridor(part['shape_id'], part['stop_bounds'])
        _corridor = fm_dict[hotspot_dict['itp_id']].corridor.copy()
        subcorridors += [_corridor]
    full_corridor = pd.concat(subcorridors).dissolve()
    fm_dict[hotspot_dict['itp_id']].add_corridor(full_corridor)
    if hotspot_dict['route_filter']:
        fm_dict[hotspot_dict['itp_id']].set_filter(route_names = hotspot_dict['route_filter'])
    else:
        fm_dict[hotspot_dict['itp_id']].reset_filter()
    fm_dict[hotspot_dict['itp_id']].corridor_metrics()
    display(fm_dict[hotspot_dict['itp_id']].quick_map_corridor())
    measured_corridor = fm_dict[hotspot_dict['itp_id']].corridor.copy()
    hotspots += [measured_corridor]
    return

In [ ]:
for hotspot in district_dict['hotspots']:
    parse_hotspot(hotspot['hotspot'])

In [ ]:
# save speedmap views across all operators for future combined map
all_speed_segs = pd.concat(hs_speed_views)
all_corr_segs = pd.concat(corr_speed_views)
# full join to preserve all unique segments, since they can differ
# from hotspot to corridor (or indeed across operators)
all_segs = all_speed_segs >> full_join(_, all_corr_segs)

In [ ]:
bb_rt_helper.bb_map_all(hotspots, corridors, all_segs, district = district_param)